In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

In [7]:
# 1. Load dataset (Assuming CSV with a column 'text')
file_path = "/content/poems.csv"  # Change this to your dataset file

df = pd.read_csv(file_path)
sequences = df['text'].apply(lambda x: [ord(c) for c in str(x)])  # Simple character encoding


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x9c in position 260: invalid start byte

In [5]:
# 2. Pad sequences to ensure uniform length
padded_sequences = pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in sequences], batch_first=True)

NameError: name 'sequences' is not defined

In [4]:
# Tokenization and Vocabulary Creation
words = " ".join(poems).split()
vocab = list(set(words))
vocab_size = len(vocab)
word_to_index = {word: i for i, word in enumerate(vocab)}
index_to_word = {i: word for i, word in enumerate(vocab)}

In [5]:
# Convert text to sequences
def text_to_sequences(text, word_to_index):
    return [[word_to_index[word] for word in sentence.split()] for sentence in text]

sequences = text_to_sequences(poems, word_to_index)

In [6]:
# One-Hot Encoding
def one_hot_encode(sequences, vocab_size):
    one_hot = []
    for seq in sequences:
        encoded_seq = []
        for index in seq:
            one_hot_vector = np.zeros(vocab_size)
            one_hot_vector[index] = 1
            encoded_seq.append(one_hot_vector)
        one_hot.append(encoded_seq)
    return torch.tensor(one_hot, dtype=torch.float32)

one_hot_sequences = one_hot_encode(sequences, vocab_size)


<ipython-input-6-ff6c36c444f1>:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  return torch.tensor(one_hot, dtype=torch.float32)


ValueError: expected sequence of length 9 at dim 1 (got 6)

In [12]:
# Define RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out


In [13]:
# Train RNN with One-Hot Encoding
def train_rnn(model, data, targets, epochs=100, lr=0.01):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")


In [14]:
# Prepare training data for one-hot encoded RNN
input_data = one_hot_sequences[:, :-1, :]
targets = torch.tensor([seq[-1] for seq in sequences], dtype=torch.long)

NameError: name 'one_hot_sequences' is not defined

In [15]:
# Initialize and train model
hidden_size = 10
model = RNNModel(vocab_size, hidden_size, vocab_size)
train_rnn(model, input_data, targets)


NameError: name 'input_data' is not defined

In [16]:
# Define LSTM Model with Trainable Word Embeddings
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

In [17]:
# Convert sequences to tensor
sequence_tensor = torch.tensor(sequences, dtype=torch.long)

ValueError: expected sequence of length 9 at dim 1 (got 6)

In [2]:
sequences_padded = pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in sequences], batch_first=True)
sequence_tensor = sequences_padded

NameError: name 'pad_sequence' is not defined

In [19]:
# Text Generation Function
def generate_text(model, seed_text, length=5):
    model.eval()
    words = seed_text.split()
    for _ in range(length):
        sequence = torch.tensor([[word_to_index[w] for w in words]], dtype=torch.long)
        with torch.no_grad():
            output = model(sequence)
            next_word_index = torch.argmax(output, dim=1).item()
            next_word = index_to_word[next_word_index]
            words.append(next_word)
    return " ".join(words)


In [20]:
# Generate text using trained LSTM model
print("Generated text:", generate_text(lstm_model, "The sun is"))

Generated text: The sun is The blue blue blue blue
